# Encoding a Large Knowledge Graph

In this notebook, we are going to walk through how to encode a large knowledge graph for the purposes of Graph RAG. We will provide two examples of how to do so, along with demonstration code.

## Example 1: Building from Already Existing Datasets

In most RAG scenarios, the subset of the information corpus that gets retrieved is crucial for whether the appropriate response to the LLM. The same is true for GNN based RAG. Consider the following dataset WebQSP:

In [ ]:
from torch_geometric.datasets import WebQSPDataset, UpdatedWebQSPDataset

# Computationally expensive, so running a small sample for now to show off the schema
# ds = WebQSPDataset('dataset')
num_questions = 100
ds = UpdatedWebQSPDataset('small_sample', limit=num_questions)

WebQSP is a dataset that is based off of a subset of the Freebase Knowledge Graph, which is an open-source knowledge graph formerly maintained by Google. For each question-answer pair in the dataset, a subgraph was chosen based on a Semantic SPARQL search on the larger knowledge graph, to provide relevent context on finding the answer. So each entry in the dataset consists of:
- A question to be answered
- The answer
- A knowledge graph subgraph of Freebase that has the context needed to answer the question.

In [ ]:
ds.raw_dataset

In [ ]:
ds.raw_dataset[0]

Although this dataset can be trained on as-is, a couple problems emerge from doing so:
1. A retrieval algorithm needs to be implemented and executed during inference time, that might not appropriately correspond to the algorithm that was used to generate the dataset subgraphs.
2. The dataset as is not stored computationally efficiently, as there will exist many duplicate nodes and edges that are shared between the questions.

As a result, it makes sense in this scenario to be able to encode all the entries into a large knowledge graph, so that duplicate nodes and edges can be avoided, and so that alternative retrieval algorithms can be tried. We can do this with the LargeGraphIndexer class:

In [ ]:
from torch_geometric.data import LargeGraphIndexer, Data, get_features_for_triplets_groups
from torch_geometric.nn.nlp import SentenceTransformer
import time
import torch
import tqdm
from itertools import chain
import networkx as nx

In [ ]:
raw_dataset_graphs = [[tuple(trip) for trip in graph] for graph in ds.raw_dataset['graph']]
print(raw_dataset_graphs[0])

To show the benefits of this indexer in action, we will use the following model to encode this sample of graphs using LargeGraphIndexer, along with naively.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SentenceTransformer(model_name='sentence-transformers/all-roberta-large-v1').to(device)
print(device)

First, we compare the clock times of encoding using both methods.

In [ ]:
# Indexing question-by-question
dataset_graphs_embedded = []
start = time.time()
for graph in tqdm.tqdm(raw_dataset_graphs):
    nodes_map = dict()
    edges_map = dict()
    edge_idx_base = []

    for src, edge, dst in graph:
        # Collect nodes
        if src not in nodes_map:
            nodes_map[src] = len(nodes_map)
        if dst not in nodes_map:
            nodes_map[dst] = len(nodes_map)
        
        # Collect edge types
        if edge not in edges_map:
            edges_map[edge] = len(edges_map)

        # Record edge
        edge_idx_base.append((nodes_map[src], edges_map[edge], nodes_map[dst]))
    
    # Encode nodes and edges
    sorted_nodes = list(sorted(nodes_map.keys(), key=lambda x: nodes_map[x]))
    sorted_edges = list(sorted(edges_map.keys(), key=lambda x: edges_map[x]))

    x = model.encode(sorted_nodes, batch_size=256)
    edge_attrs_map = model.encode(sorted_edges, batch_size=256)
    
    edge_attrs = []
    edge_idx = []
    for trip in edge_idx_base:
        edge_attrs.append(edge_attrs_map[trip[1]])
        edge_idx.append([trip[0], trip[2]])

    dataset_graphs_embedded.append(Data(x=x, edge_index=torch.tensor(edge_idx).T, edge_attr=torch.stack(edge_attrs, dim=0)))
    
    
print(time.time()-start)

In [ ]:
# Using LargeGraphIndexer to make one large knowledge graph
from torch_geometric.data.large_graph_indexer import EDGE_RELATION

start = time.time()
all_triplets_together = chain.from_iterable(raw_dataset_graphs)
# Index as one large graph
print('Indexing...')
indexer = LargeGraphIndexer.from_triplets(all_triplets_together)

# first the nodes
unique_nodes = indexer.get_unique_node_features()
node_encs = model.encode(unique_nodes, batch_size=256)
indexer.add_node_feature(new_feature_name='x', new_feature_vals=node_encs)

# then the edges
unique_edges = indexer.get_unique_edge_features(feature_name=EDGE_RELATION)
edge_attr = model.encode(unique_edges, batch_size=256)
indexer.add_edge_feature(new_feature_name="edge_attr", new_feature_vals=edge_attr, map_from_feature=EDGE_RELATION)

ckpt_time = time.time()
whole_knowledge_graph = indexer.to_data(node_feature_name='x', edge_feature_name='edge_attr') 
whole_graph_done = time.time()
print(f"Time to create whole knowledge_graph: {whole_graph_done-start}")

# Compute this to make sure we're comparing like to like on final time printout
whole_graph_diff = whole_graph_done-ckpt_time

# retrieve subgraphs
print('Retrieving Subgraphs...')
dataset_graphs_embedded_largegraphindexer = [graph for graph in tqdm.tqdm(get_features_for_triplets_groups(indexer=indexer, triplet_groups=raw_dataset_graphs), total=num_questions)]
print(time.time()-start-whole_graph_diff)

The large graph indexer allows us to compute the entire knowledge graph from a series of samples, so that new retrieval methods can also be tested on the entire graph. We will see this attempted in practice later on.

It's worth noting that, although the times are relatively similar right now, the speedup with largegraphindexer will be much higher as the size of the knowledge graph grows. This is due to the speedup being a factor of the number of unique nodes and edges in the graph.

In [ ]:
dataset_graphs_embedded_largegraphindexer

In [ ]:
dataset_graphs_embedded

We expect the two results to be functionally identical, with the differences being due to floating point jitter.

In [ ]:
def results_are_close_enough(ground_truth: Data, new_method: Data, thresh=.8):
    def _sorted_tensors_are_close(tensor1, tensor2):
        return torch.all(torch.isclose(tensor1.sort(dim=0)[0], tensor2.sort(dim=0)[0]).float().mean(axis=1) > thresh)
    def _graphs_are_same(tensor1, tensor2):
        return nx.weisfeiler_lehman_graph_hash(nx.Graph(tensor1.T)) == nx.weisfeiler_lehman_graph_hash(nx.Graph(tensor2.T))
    return _sorted_tensors_are_close(ground_truth.x, new_method.x) \
        and _sorted_tensors_are_close(ground_truth.edge_attr, new_method.edge_attr) \
        and _graphs_are_same(ground_truth.edge_index, new_method.edge_index)

In [ ]:
all_results_match = True
for old_graph, new_graph in tqdm.tqdm(zip(dataset_graphs_embedded, dataset_graphs_embedded_largegraphindexer), total=num_questions):
    all_results_match &= results_are_close_enough(old_graph, new_graph)
all_results_match

When scaled up to the entire dataset, we see a 2x speedup with indexing this way.

WebQSPDataset is a question-by-question implementation.

UpdatedQSPDataset is a LargeGraphIndexer implementation.

These were computed on an RTX 4090 with 24GB of memory. Your milage may vary.

In [ ]:
demo_number_of_graphs = 4700 # out of 4700

In [ ]:
# Note: WebQSP doesn't let you limit the number of questions, so we compute the whole dataset and estimate the time by assuming each question takes the same amount of time
start = time.time()
ds_basic = WebQSPDataset('basic_dataset')
web_qsp_time = (time.time() - start)*demo_number_of_graphs/4700
print(time.time()-start)

In [ ]:
start = time.time()
ds_updated = UpdatedWebQSPDataset('updated_dataset', limit=demo_number_of_graphs)
updated_time = time.time() - start
print(time.time()-start)

In [ ]:
import matplotlib.pyplot as plt
plt.bar(['WebQSP', 'UpdatedQSP'], [web_qsp_time,, updated_time])

## Example 2: Building a new Dataset from Questions and an already-existing Knowledge Graph

In this example, we will take a set of multi-hop questions, and combine them with an existing Wikidata knowledge graph to produce a new dataset.

To be continued in 0.1